# Interactivity with Plotly

<img src="https://camo.githubusercontent.com/a38f9ed71170e3112dd3e4af1d11d4b081fe2e25/687474703a2f2f692e696d6775722e636f6d2f643379346e776d2e676966" alt="Drawing" style="width: 800px;"/>

<br>
For the most part, interactivity in plotly will be done through javascript. However, plotly.py and jupyter notebooks do have enough interactive tools to create a simple plot with one. This may be useful for creating plots in the first few weeks of the project or testing your physics code as you go along.<br><br>

for now let's import all the module's we will need


In [ ]:
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np

## Why (and why not) use Jupyter Interacts


Now the way that plotly's iplot function works is that it sends off all of the input data and layout dictionaries to a standalone html. This means that - in general - you cannot send new data to an already existing iplot. So if you want to update the data of an iplot using jupyter, what you will have to do is replot the graph every time you change something.
<br><br>
To do this we can use jupyter's interact() widget, which takes a function and its respective variables as inputs. The interact function then produces either sliders or buttons to cycle through the variables of the input function. An example of using interact to change the data of a plotly graph is shown below:

In [ ]:
def plot(x):
    global data
    data = []
    trace1 = dict(
    go.Scatter(
        x=[1,2,4,x],
        y=[5,0,8,2],
        mode='markers',
        name='bubbles!',
        marker=dict(
            size=15
        )
        
    )
    )

    trace2 = dict(
    go.Scatter(
        x=[1,3,5,7],
        y=[1,3,5,7],
        name='lines',
        line = dict(
            width=2,
            color='rgb(21,117,22)'
        )
        
    )
    )
    data = [trace1,trace2]
    iplot(data)
    
interact(plot,x=widgets.IntSlider(min=0,max=10,step=1,value=0))


## Why (and why not use) Plotly.py's interacts

As well as jupyter's built in ipython widgets and interacts, plotly has it's own set of interactivities. These interactivities can be split into three main groups: animation, sliders, and buttons. What's more, none of them require you to replot the entire graph after changing one variable!!

### Let's start with a quick tutorial on buttons:

For all three of these interactivities we will usually start off by creating a dictionary called updatemenus in your layout dictionary. This updatemenus dict is what contains the information about whatever widget you want to impliment.<br><br>
For a button, we simply create a dict called buttons that contains all of the different datasets or layout types that we would like to eventually plot. Say if we want to switch between plotting a sphere or a cube; with a plotly button this is possible. <br><br>
The most important atribute of the buttons dict is the 'args' attribute, and tells plotly what attribute of your figure you want to change. In the example below, we are changing the x coordinate of the data. However, in general we write args in the form 'args':<br> [{'attribute': new_value_of_attribute,'attribute2: new_value_of_attribute2'}]

In [ ]:
x = [2]
t= np.linspace(0,2,50)
y = [0]


data=[dict(x=x, y=y,
           name='frame',
           mode='markers', 
           line=dict(width=2, color='blue')),

    ]

layout=dict(width=1000, height=500,
            xaxis=dict(range=[0, 2*np.pi], autorange=False, zeroline=False),
            yaxis=dict(range=[-1.1, 1.1], autorange=False, zeroline=False),
            title='Button', hovermode='closest',
            updatemenus= [
                {
                    'buttons':[
                        {
                            'args': [{'x':[[2]]}],
                            'label': 'frame1',
                            'method': 'update'
                        },
                        {
                            'args': [{'x':[[4]]}],
                            'label': 'frame2',
                            'method': 'update'
                        }
                    ]
                }
            ])
 
figure = dict(data=data,layout=layout)
iplot(figure)


### Now for the animations:

The way that animating in plotly works is that - as well as inputing data and layout dictionaries into your iplot function - you must input a dictionary called 'frames'. <br><br>
Now 'frames' is essentially just a series of other figures (data and layout dictionaries) that the plot will cycle through. These figures can have different data and layout than your initial data and layout inputs, so when you cycle through them it looks like the graph is changing and animating.<br><br>
You will also need the standard updatemenus dictionary in your initial layout input. The updatemenus should contain two buttons that start and stop the animation, using the method 'animate', rather than update.<br>
Below is an example that displays eveything we've discussed concerning animation:

In [ ]:
x = [1]
t= np.linspace(0,2,50)
y = [-1]


data=[dict(x=x, y=y,
           name='frame',
           mode='markers', 
           line=dict(width=2, color='blue')),

    ]

#asdasdasd
#hi 

layout=dict(width=1000, height=500,
            xaxis=dict(range=[0, 2*np.pi], autorange=False, zeroline=False),
            yaxis=dict(range=[-1.1, 1.1], autorange=False, zeroline=False),
            title='Animation', hovermode='closest',
            updatemenus= [
                {
                    'buttons':[
                        {
                            'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                            'transition': {'duration': 0}}],
                            'label': 'Pause',
                            'method': 'animate'
                        },
                        {
                            'args': [None, {'frame': {'duration': 0, 'redraw': False},
                                'fromcurrent': True, 'transition': {'duration': 0, 'easing': 'quadratic-in-out'}}],
                            'label': 'Play',
                            'method': 'animate'
                        }
                    ]
                }
            ])

frames=[dict(data=[dict(x=x+2*t[k], 
                        y=y+t[k]), 

                  ]) for k in range(50)]    
figure = dict(data=data,layout=layout,frames=frames)
iplot(figure)


### Finally the sliders:

Finally we have sliders, which are half way inbetween buttons and animations. When adding sliders to plotly, rather than using updatemenus we have a dictionary (still within the layout dictionary) called sliders.<br><br>

From the example below you can see that this sliders dictionary has a number of different attributes, but our main focus is the steps attribute. Steps takes the form of a list of all the dataframes we want to plot. Here we have created steps using a simple for loop, which is often the simplest solution.

In [ ]:
slider_range=6
steps=[]
for i in range(0,slider_range,1):
    step = dict(
        method='update',
        args=[{'x':[[i]]}]
    )
    steps.append(step)
    
sliders= [dict(
    active=2,
    currentvalue={'prefix':'x position'},
    pad={'t':slider_range},
    steps=steps
)]

data=[dict(x=[2], y=[0],
           name='frame',
           mode='markers', 
           line=dict(width=2, color='blue')),

    ]

layout=dict(width=1000, height=500,
            xaxis=dict(range=[0, 2*np.pi], autorange=False, zeroline=False),
            yaxis=dict(range=[-1.1, 1.1], autorange=False, zeroline=False),
            title='Slider', hovermode='closest',
            sliders=sliders)
figure = dict(data=data,layout=layout)
iplot(figure)

### BE AWARE: We will generally not be using plotly's interactivities because in all the three methods we have looked at, you must send of all of your frames at once! When you move a slider or press a button you are simply picking between data that you have already created. So if we ever want to continously add data, or use multiple sliders for the same plot, we will have to use javascript!